In [ ]:
import numpy as np
from matplotlib import pyplot as plt, patches
import imageio.v2 as imageio

from qiskit import QuantumCircuit, transpile, Aer, execute
from qiskit.circuit.library import HamiltonianGate

In [ ]:
def generate_sierpinski_triangle(points, ternaries, depth, unique_points):
    if depth == 0:
        for point, ternary in zip(points, ternaries):
            if point in unique_points:
                unique_points[point].append(ternary)
            else:
                unique_points[point] = [ternary]
    else:
        # Calculate midpoints and their ternary representations
        mid1 = ((points[0][0] + points[1][0]) / 2, (points[0][1] + points[1][1]) / 2)
        mid2 = ((points[1][0] + points[2][0]) / 2, (points[1][1] + points[2][1]) / 2)
        mid3 = ((points[0][0] + points[2][0]) / 2, (points[0][1] + points[2][1]) / 2)

        # Recurse for each sub-triangle
        generate_sierpinski_triangle([points[0], mid1, mid3], [ternaries[0]+'0', ternaries[0]+'1', ternaries[0]+'2'], depth - 1, unique_points)
        generate_sierpinski_triangle([mid1, points[1], mid2], [ternaries[1]+'0', ternaries[1]+'1', ternaries[1]+'2'], depth - 1, unique_points)
        generate_sierpinski_triangle([mid3, mid2, points[2]], [ternaries[2]+'0', ternaries[2]+'1', ternaries[2]+'2'], depth - 1, unique_points)

def ternary_differs_by_one(a, b):
    """Check if two ternary numbers differ by exactly one digit."""
    count_diff = 0
    for digit_a, digit_b in zip(a, b):
        if digit_a != digit_b:
            count_diff += 1
    return count_diff == 1

def is_adjacent_ternary(ternaries_a, ternaries_b):
    """Check if two points are adjacent."""
    for a in ternaries_a:
        for b in ternaries_b:
            if ternary_differs_by_one(a, b):
                return True
    return False

def is_adjacent_point(point_a, point_b, depth):
    """Check if two points are adjacent."""
    threshold = 1.1 / (2 ** depth)
    return np.linalg.norm(np.array(point_a) - np.array(point_b)) < threshold

In [ ]:
# Initial triangle points and their ternary representations
initial_points = [(0, 0), (1, 0), (0.5, np.sqrt(0.75))]  # Vertices of the triangle
initial_ternaries = ['0', '1', '2']  # Ternary representations

# Generate points and ternaries
max_depth = 4  # Adjust the depth as needed
unique_points = {}
generate_sierpinski_triangle(initial_points, initial_ternaries, max_depth, unique_points)

# # Print unique points and their ternary representations
# for point, ternary in unique_points.items():
#     print(f"Point: {point}, Ternary: {ternary}")
    
# Using the unique_points dictionary from the previous step
point_list = list(unique_points.keys())
ternary_list = list(unique_points.values())
num_points = len(point_list)

# Initialize adjacency matrix
adjacency_matrix = np.zeros((num_points, num_points))

# Calculate the adjacency matrix
C = 0.5
for i in range(num_points):
    for j in range(i + 1, num_points):
        if is_adjacent_ternary(ternary_list[i], ternary_list[j]) and is_adjacent_point(point_list[i], point_list[j], max_depth):
            adjacency_matrix[i][j] = adjacency_matrix[j][i] = C

# Fill diagonal with beta
beta = 0.5
np.fill_diagonal(adjacency_matrix, beta)

In [ ]:
# Calculate the number of qubits needed for the representation
npoint = adjacency_matrix.shape[0]
nqubit = int(np.ceil(np.log2(npoint)))
nstate = int(2**nqubit)
print("Adjacency matrix:")
print(adjacency_matrix.shape)

# Construct the Hamiltonian
H = np.zeros((nstate, nstate))
H[np.ix_(np.arange(npoint), np.arange(npoint))] = adjacency_matrix
assert np.allclose(H, np.conj(H.T))
print("Hamiltonian H:")
print(H.shape)

def vizp(nodestate, points, depth, path=None):
    # calculate the possibility of each node
    pnode = np.array(list(nodestate.values()))
    ampli_qubit = np.sqrt(pnode)

    radius = 1.0 / (2 ** depth) / 2
    npoint = len(points)

    fig, ax = plt.subplots(1, 1)
    # draw state vector
    for i in range(npoint):
        circleInt = patches.Circle((points[i][0], points[i][1]), ampli_qubit[i]*radius, color='r',alpha=1.0)
        ax.add_patch(circleInt)
    # draw connections
    for i in range(npoint):
        for j in range(i + 1, npoint):
            if adjacency_matrix[i, j] > 0:
                ax.plot([points[i][0], points[j][0]], [points[i][1], points[j][1]], 'b-', alpha=0.1)  # Line
    ax.set_aspect('equal')
    ax.axis('off')
    if path is not None:
        plt.savefig(path, dpi=100)
        plt.close()
    else:
        plt.show()

In [ ]:
# Create the circuits
circuits = []
# Time parameter for evolution
for t in np.arange(0, 20.1, 0.1):
    # Qiskit simulation
    qc = QuantumCircuit(nqubit)
    U = HamiltonianGate(H, time=t)
    qc.append(U, qc.qubits)

    circuits.append(qc)

# Try backend with local simulator
backend = Aer.get_backend('statevector_simulator')

# Transpile circuits for the backend
transpiled_circuits = transpile(circuits, backend)

# Execute all the circuits as one job
job = execute(transpiled_circuits, backend, shots=129536)

# Retrieve results for each circuit
results = {f'circuit_{i}': job.result().get_counts(circuit) for i, circuit in enumerate(circuits)}

In [ ]:
from qiskit import IBMQ

IBMQ.save_account('0fd555976da334bbb68f4163965c75acec0582cf738486d0c4004a80f54b5db7ce978e162531587b9d134b3ee78ea21f1687a782add3a37a620f438cb51e3628')

In [ ]:
provider = IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')
available_cloud_backends = provider.backends()
print('\n Cloud backends:')
for i in available_cloud_backends: print(i)
available_local_backends = Aer.backends()
print('\n Local backends: ')
for i in available_local_backends: print(i)

In [64]:
from qiskit.tools.monitor import job_monitor

# Create the circuits
circuits = []
# Time parameter for evolution
for t in [0.0, 0.1, 0.5, 1.0, 5.0, 10.0]:
    # Qiskit simulation
    qc = QuantumCircuit(nqubit)
    U = HamiltonianGate(H, time=t)
    qc.append(U, qc.qubits)
    qc.measure_all()

    circuits.append(qc)

#  Get backend of actual quantum computer
backend = provider.get_backend('ibm_brisbane')

# Transpile circuits for the backend
transpiled_circuits = transpile(circuits, backend)

In [66]:
# Execute all the circuits as one job
job = execute(transpiled_circuits, backend, shots=1024)

# Monitor job status
job_monitor(job)

# Retrieve results for each circuit
results = {f'circuit_{i}': job.result().get_counts(circuit) for i, circuit in enumerate(circuits)}

# Print results
for i in range(len(results)):
    print(f"Time: {i * 0.1}")
    print(results[f'circuit_{i}'])
    print()

Job Status: job is queued (None)

In [ ]:
# Generate key lists as binary string from 0 to 2**nqubit
key_list = []
for i in range(2**nqubit):
    key_list.append(format(i, '0'+str(nqubit)+'b'))

# Reformatting the results
for result in results:
    for key in key_list:
        if key not in results[result]:
            results[result][key] = 0.0

# Plot the results
filenames = []
for i in np.arange(len(circuits)):
    filename = './Figs/fractal_cloud/%d.png' % i
    filenames.append(filename)
    vizp(results[f'circuit_{i}'], point_list, max_depth, filename)

# Create a GIF
with imageio.get_writer('./Figs/fractal_cloud/cloud.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)